In [5]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [6]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = '10.10.100.47'
DB2_PORT = '53000'
DB2_USERNAME = 'db2inst1'
DB2_PASSWORD = 'db2inst1'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

In [7]:
# Region
cols_map = [
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LOG_DATE', 'des_col': 'log_date', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COUNTER_ID', 'des_col': 'counter_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MAX_AMOUNT', 'des_col': 'max_amount', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'AMOUNT_PAID', 'des_col': 'amount_paid', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'PENDING_CNT', 'des_col': 'pending_cnt', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'APPROVED_CNT', 'des_col': 'approved_cnt', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'RECORD_STATUS', 'des_col': 'record_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CHECK_CONDITION', 'des_col': 'check_condition', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATE_DATE', 'des_col': 'create_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'CREATED_BY', 'des_col': 'created_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DB_IP_ADDRESS', 'des_col': 'db_ip_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG1', 'des_col': 'flag1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG2', 'des_col': 'flag2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG3', 'des_col': 'flag3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG4', 'des_col': 'flag4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG5', 'des_col': 'flag5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MODULE_ID', 'des_col': 'module_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MONTH', 'des_col': 'month', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'}, 
    {'src_col': 'YEAR', 'des_col': 'year', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'SCREEN_ID', 'des_col': 'screen_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SOURCE_OFFICE_ID', 'des_col': 'source_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBMODULE_ID', 'des_col': 'submodule_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TARGET_OFFICE_ID', 'des_col': 'target_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATE_DATE', 'des_col': 'update_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'UPDATED_BY', 'des_col': 'updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'EXTENDED_MAXAMOUNT', 'des_col': 'extended_maxamount', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'WEEKLY_MAX_AMOUNT', 'des_col': 'weekly_max_amount', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'WEEKLY_PAYMENT_AMOUNT', 'des_col': 'weekly_payment_amount', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'MONTHLY_MAX_AMOUNT', 'des_col': 'monthly_max_amount', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'MONTHLY_PAYMENT_AMOUNT', 'des_col': 'monthly_payment_amount', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'VALIDATION_TYPE', 'des_col': 'validation_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'WEEK_END_DATE', 'des_col': 'week_end_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'MONTH_END_DATE', 'des_col': 'month_end_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'WEEKLY_EXTENDED_MAXAMOUNT', 'des_col': 'weekly_extended_maxamount', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MONTHLY_EXTENDED_MAXAMOUNT', 'des_col': 'monthly_extended_maxamount', 'src_dtype': 'String', 'des_dtype': 'String'}
]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'DABS    .COUNTER_MAXAMOUNT', 'public', 'counter_maxamount', cols_map)
print("Total Migrated Records (counter_maxamount): " + str(migrated_row_count))


Total Migrated Records (counter_maxamount): Total Src Records: 282 Total migrated Records to dest: 282
